## ThinkDSP

This notebook contains code examples from Chapter 4: Noise

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
from __future__ import print_function, division

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [4]:
import thinkdsp
import thinkplot
import thinkstats2 

import numpy as np

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

SyntaxError: from __future__ imports must occur at the beginning of the file (thinkplot.py, line 11)

In [ ]:
thinkdsp.random_seed(17)

The simplest noise to generate is uncorrelated uniform (UU) noise:

In [ ]:
signal = thinkdsp.UncorrelatedUniformNoise()
wave = signal.make_wave(duration=1, framerate=11025)
wave.make_audio()

Here's what a segment of it looks like:

In [ ]:
segment = wave.segment(duration=0.1)
segment.plot(linewidth=1)
thinkplot.config(xlabel='time',
                 ylabel='amplitude',
                 ylim=[-1.05, 1.05],
                 legend=False)

And here's the spectrum:

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot(linewidth=0.5)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='amplitude',
                 xlim=[0, spectrum.fs[-1]])

In the context of noise it is more conventional to look at the spectrum of power, which is the square of amplitude:

In [ ]:
spectrum.plot_power(linewidth=0.5)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xlim=[0, spectrum.fs[-1]])

UU noise has the same power at all frequencies, on average, which we can confirm by looking at the normalized cumulative sum of power, which I call an integrated spectrum:

In [ ]:
integ = spectrum.make_integrated_spectrum()
integ.plot_power()
thinkplot.config(xlabel='frequency (Hz)',
                ylabel='cumulative power',
                xlim=[0, spectrum.fs[-1]])

A straight line in this figure indicates that UU noise has equal power at all frequencies, on average.  By analogy with light, noise with this property is called "white noise".

### Brownian noise

Brownian noise is generated by adding up a sequence of random steps.

In [ ]:
signal = thinkdsp.BrownianNoise()
wave = signal.make_wave(duration=0.5, framerate=11025)
wave.make_audio()

The sound is less bright, or more muffled, than white noise.

Here's what the wave looks like:

In [ ]:
wave.plot(linewidth=1)
thinkplot.config(xlabel='time',
                 ylabel='amplitude',
                 ylim=[-1.05, 1.05])

Here's what the power spectrum looks like on a linear scale.

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot_power(linewidth=0.5)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xlim=[0, spectrum.fs[-1]])

So much of the energy is at low frequencies, we can't even see the high frequencies.

We can get a better view by plotting the power spectrum on a log-log scale.

In [ ]:
# The f=0 component is very small, so on a log scale
# it's very negative.  If we clobber it before plotting,
# we can see the rest of the spectrum better.
spectrum.hs[0] = 0

spectrum.plot_power(linewidth=0.5)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xscale='log',
                 yscale='log',
                 xlim=[0, spectrum.fs[-1]])

Now the relationship between power and frequency is clearer.  The slope of this line is approximately -2, which indicates that $P = K / f^2$, for some constant $K$.

In [ ]:
signal = thinkdsp.BrownianNoise()
wave = signal.make_wave(duration=0.5, framerate=11025)
spectrum = wave.make_spectrum()
result = spectrum.estimate_slope()
result.slope

The estimated slope of the line is closer to -1.8 than -2, for reasons we'll see later.

### Pink noise

Pink noise is characterized by a parameter, $\beta$, usually between 0 and 2.  You can hear the differences below.

With $\beta=0$, we get white noise:

In [ ]:
signal = thinkdsp.PinkNoise(beta=0)
wave = signal.make_wave(duration=0.5)
wave.make_audio()

With $\beta=1$, pink noise has the relationship $P = K / f$, which is why it is also called $1/f$ noise.

In [ ]:
signal = thinkdsp.PinkNoise(beta=1)
wave = signal.make_wave(duration=0.5)
wave.make_audio()

With $\beta=2$, we get Brownian (aka red) noise.

In [ ]:
signal = thinkdsp.PinkNoise(beta=2)
wave = signal.make_wave(duration=0.5)
wave.make_audio()

The following figure shows the power spectrums for white, pink, and red noise on a log-log scale.

In [ ]:
colors = ['#9ecae1', '#4292c6', '#2171b5']
betas = [0, 1, 2]

for beta, color in zip(betas, colors):
    signal = thinkdsp.PinkNoise(beta=beta)
    wave = signal.make_wave(duration=0.5, framerate=1024)
    spectrum = wave.make_spectrum()
    spectrum.hs[0] = 0
    spectrum.plot_power(linewidth=1, color=color)
    
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xscale='log',
                 yscale='log',
                 xlim=[0, spectrum.fs[-1]])

### Uncorrelated Gaussian noise

An alternative to UU noise is uncorrelated Gaussian (UG noise).

In [ ]:
signal = thinkdsp.UncorrelatedGaussianNoise()
wave = signal.make_wave(duration=0.5, framerate=11025)
wave.plot(linewidth=0.5)
thinkplot.config(xlabel='time',
                 ylabel='amplitude')

The spectrum of UG noise is also UG noise.

In [ ]:
spectrum = wave.make_spectrum()
spectrum.plot_power(linewidth=1)
thinkplot.config(xlabel='frequency (Hz)',
                 ylabel='power',
                 xlim=[0, spectrum.fs[-1]])

We can use a normal probability plot to test the distribution of the power spectrum.

In [ ]:
from thinkstats2 import NormalProbabilityPlot

NormalProbabilityPlot(spectrum.real, label='real part')
thinkplot.config(xlabel='normal sample',
                 ylabel='power',
                 ylim=[-250, 250],
                 legend=True,
                 loc='lower right')

A straight line on a normal probability plot indicates that the distribution of the real part of the spectrum is Gaussian.

In [ ]:
NormalProbabilityPlot(spectrum.imag, label='imag part')
thinkplot.config(xlabel='normal sample',
                 ylabel='power',
                 ylim=[-250, 250],
                 legend=True,
                 loc='lower right')

And so is the imaginary part.